## Preprocessing

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

from tensorflow.keras.layers import Dense
from tensorflow import keras
from keras_tuner import RandomSearch, Hyperband

import os


#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df= application_df.drop(["EIN", "NAME","SPECIAL_CONSIDERATIONS"], axis = 1)

In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_counts=application_df["APPLICATION_TYPE"].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
application_types_to_replace = (list(application_counts[application_counts < 500].index))
application_types_to_replace


['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [10]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = (list(application_counts[application_counts < 500].index))

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning
class_count = application_df["CLASSIFICATION"].value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [12]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count[class_count > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [13]:
classifications_to_replace = list(class_count[class_count < 1000].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
dummified_df = pd.get_dummies(application_df, dtype=int,drop_first= True)

In [15]:

# Split our preprocessed data into our features and target arrays
X = dummified_df.drop("IS_SUCCESSFUL", axis=1)
y = dummified_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)



In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



## Compile, Train and Evaluate the Model

In [17]:


# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(Dense(units = 20, activation = "relu", input_dim = X_train.shape[1]))

# Second hidden layer
nn2.add(Dense(units = 80, activation = "sigmoid"))


nn2.add(Dense(units = 200, activation = "relu"))


nn2.add(Dense(units = 500, activation = "sigmoid"))

# Output layer
nn2.add(Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn2.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                720       
                                                                 
 dense_1 (Dense)             (None, 80)                1680      
                                                                 
 dense_2 (Dense)             (None, 200)               16200     
                                                                 
 dense_3 (Dense)             (None, 500)               100500    
                                                                 
 dense_4 (Dense)             (None, 1)                 501       
                                                                 
Total params: 119601 (467.19 KB)
Trainable params: 119601 (467.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Compile the model
nn2.compile(loss = "binary_crossentropy", optimizer= keras.optimizers.Adam(learning_rate=0.01), metrics = ["accuracy"])

In [19]:
# Train the model
nn2.fit(X_train_scaled, y_train, epochs = 35)

Epoch 1/35
804/804 [==============================] - 4s 4ms/step - loss: 0.5966 - accuracy: 0.7021
Epoch 2/35
804/804 [==============================] - 5s 6ms/step - loss: 0.5630 - accuracy: 0.7262
Epoch 3/35
804/804 [==============================] - 3s 4ms/step - loss: 0.5567 - accuracy: 0.7284
Epoch 4/35
804/804 [==============================] - 3s 4ms/step - loss: 0.5557 - accuracy: 0.7297
Epoch 5/35
804/804 [==============================] - 3s 4ms/step - loss: 0.5527 - accuracy: 0.7297
Epoch 6/35
804/804 [==============================] - 5s 6ms/step - loss: 0.5533 - accuracy: 0.7274
Epoch 7/35
804/804 [==============================] - 3s 4ms/step - loss: 0.5525 - accuracy: 0.7297
Epoch 8/35
804/804 [==============================] - 3s 4ms/step - loss: 0.5510 - accuracy: 0.7305
Epoch 9/35
804/804 [==============================] - 3s 4ms/step - loss: 0.5517 - accuracy: 0.7301
Epoch 10/35
804/804 [==============================] - 5s 6ms/step - loss: 0.5507 - accuracy: 0.7289

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5589 - accuracy: 0.7240 - 655ms/epoch - 2ms/step
Loss: 0.5589169263839722, Accuracy: 0.7239649891853333


In [22]:
# Export our model to HDF5 file
nn2.save("AlphabetSoupCharity_Optimization2.h5")

In [ ]:
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: /content
